# Merchant Churn Prediction – Modeling Notebook (Local Python + Snowflake Connector)

This notebook assumes:

1. You already created an **Analytical Base Table** (ABT) in Snowflake named `CHURN_MODEL_ABT`.
2. Each row represents **one merchant at a snapshot date**.
3. There is a binary target column, e.g. `IS_CHURNED_TARGET` (1 = churned, 0 = active).
4. You have a `MERCHANT_ID` column used only as an identifier (not a feature).

We will:
- Connect to Snowflake (locally) using `snowflake-connector-python`
- Load the ABT into pandas
- Perform a time-based train/test split
- Train a baseline Logistic Regression model
- Train an XGBoost model
- Evaluate with ROC-AUC and a simple lift analysis
- Export a ranked churn risk report


## 1. Install and import dependencies

Run this cell once to install required libraries (if needed). Comment out `pip` lines once installed.


In [ ]:
# !pip install snowflake-connector-python pandas scikit-learn xgboost matplotlib

import os
import pandas as pd
import numpy as np
import snowflake.connector
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import matplotlib.pyplot as plt


## 2. Configure Snowflake connection

Update the placeholders (`YOUR_...`) with your real values.
Use environment variables in real projects for security.


In [ ]:
SNOWFLAKE_CONFIG = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT", "YOUR_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER", "YOUR_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD", "YOUR_PASSWORD"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE", "YOUR_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE", "YOUR_DB"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA", "YOUR_SCHEMA"),
    "role": os.getenv("SNOWFLAKE_ROLE", "YOUR_ROLE"),
}

def get_snowflake_connection():
    conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
    return conn


## 3. Load ABT (`CHURN_MODEL_ABT`) from Snowflake

Assumptions:
- Table name: `CHURN_MODEL_ABT`
- Contains at least:
  - `MERCHANT_ID`
  - `IS_CHURNED_TARGET` (0/1)
  - Optional `SNAPSHOT_DATE` (for time-based split)
  - Many engineered feature columns


In [ ]:
conn = get_snowflake_connection()
query = "SELECT * FROM CHURN_MODEL_ABT"
df = pd.read_sql(query, conn)
conn.close()

print("Shape of ABT:", df.shape)
df.head()

## 4. Basic sanity checks

Check target balance and missing values.


In [ ]:
target_col = "IS_CHURNED_TARGET"  # adjust if different
id_cols = ["MERCHANT_ID"]  # adjust if you have more IDs

print(df[target_col].value_counts(normalize=True).rename("ratio"))
print("\nMissing values per column (top 20):")
print(df.isna().mean().sort_values(ascending=False).head(20))


## 5. Train/Test split (time-based if possible)

If you have a `SNAPSHOT_DATE` column, we do a **temporal split**.
Otherwise, we use a simple random split.


In [ ]:
date_col = "SNAPSHOT_DATE" if "SNAPSHOT_DATE" in df.columns else None

feature_cols = [c for c in df.columns if c not in id_cols + [target_col]]

if date_col:
    df[date_col] = pd.to_datetime(df[date_col])
    cutoff_date = df[date_col].quantile(0.8)  # 80% train, 20% test by time
    train_df = df[df[date_col] <= cutoff_date].copy()
    test_df = df[df[date_col] > cutoff_date].copy()
else:
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

X_train = train_df[feature_cols]
y_train = train_df[target_col]
X_test = test_df[feature_cols]
y_test = test_df[target_col]

print("Train shape:", X_train.shape, " Test shape:", X_test.shape)

## 6. Handle class imbalance (optional)

We start simple by using `class_weight='balanced'` in Logistic Regression.
In later iterations, you can explore SMOTE or focal loss.


In [ ]:
# Simple numeric-only filter (if you have categorical features, encode them first)
X_train_num = X_train.select_dtypes(include=["number"]).fillna(0)
X_test_num = X_test[X_train_num.columns].fillna(0)

print("Numeric feature count:", X_train_num.shape[1])

## 7. Baseline model – Logistic Regression


In [ ]:
lr = LogisticRegression(max_iter=1000, class_weight="balanced")
lr.fit(X_train_num, y_train)
y_pred_proba_lr = lr.predict_proba(X_test_num)[:, 1]

roc_auc_lr = roc_auc_score(y_test, y_pred_proba_lr)
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba_lr)
pr_auc_lr = auc(recall, precision)

print(f"Logistic Regression ROC-AUC: {roc_auc_lr:.3f}")
print(f"Logistic Regression PR-AUC:  {pr_auc_lr:.3f}")

## 8. XGBoost model


In [ ]:
xgb = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method="hist",
)

xgb.fit(X_train_num, y_train)
y_pred_proba_xgb = xgb.predict_proba(X_test_num)[:, 1]

roc_auc_xgb = roc_auc_score(y_test, y_pred_proba_xgb)
precision_xgb, recall_xgb, _ = precision_recall_curve(y_test, y_pred_proba_xgb)
pr_auc_xgb = auc(recall_xgb, precision_xgb)

print(f"XGBoost ROC-AUC: {roc_auc_xgb:.3f}")
print(f"XGBoost PR-AUC:  {pr_auc_xgb:.3f}")

## 9. Simple lift analysis

We rank merchants by predicted churn risk and see how many churners
are captured in the top X% of the population.


In [ ]:
results = test_df[id_cols + [target_col]].copy()
results["score_xgb"] = y_pred_proba_xgb
results = results.sort_values("score_xgb", ascending=False).reset_index(drop=True)

results["decile"] = pd.qcut(results.index, 10, labels=False) + 1  # 1 = top 10%

lift_table = results.groupby("decile").agg(
    merchants=(target_col, "count"),
    churners=(target_col, "sum"),
).reset_index()

lift_table["churn_rate"] = lift_table["churners"] / lift_table["merchants"]

print(lift_table)


## 10. Export ranked churn risk report

This CSV can be shared with business/AM teams for validation.


In [ ]:
output_cols = id_cols + [target_col, "score_xgb"]
ranked_output = results[output_cols].sort_values("score_xgb", ascending=False)
output_path = "churn_risk_scores_xgb.csv"
ranked_output.to_csv(output_path, index=False)
output_path

## 11. (Optional) Save the trained model

In a real project, you would save the model to disk or a model registry
for use in batch scoring or online inference.


In [ ]:
import joblib

joblib.dump(xgb, "xgb_churn_model.pkl")
"xgb_churn_model.pkl"